# MNIST Using Tensorflow + Keras

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
import keras

import numpy as np
%matplotlib inline

Using TensorFlow backend.


In [20]:
# Data Load
x_minst = np.load('train_X.npy')
y_minst = np.load('train_y.npy')
kaggle_test = np.load('test_X.npy')

In [21]:
from sklearn.model_selection import train_test_split
# Train Dataset
x_train_mnist, x_test_mnist, y_train_mnist, y_test_mnist = train_test_split(x_minst, y_minst, test_size=0.20, random_state=0)

In [22]:
# Data Shape Change
x_train_mnst_4d = x_train_mnist.reshape(x_train_mnist.shape[0], 64, 64, 1)
x_test_mnst_4d = x_test_mnist.reshape(x_test_mnist.shape[0], 64, 64, 1)

In [23]:
x_train_mnst_4d = x_train_mnst_4d.astype('float32')
x_test_mnst_4d = x_test_mnst_4d.astype('float32')
x_train_mnst_4d /= 255.
x_test_mnst_4d /= 255.
print('x_train shape:', x_train_mnst_4d.shape)
print(x_train_mnst_4d.shape[0], 'train samples')
print(x_test_mnst_4d.shape[0], 'test samples')

x_train shape: (28000, 64, 64, 1)
28000 train samples
7000 test samples


In [24]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train_mnist, 10)
y_test = keras.utils.to_categorical(y_test_mnist, 10)
input_shape = (64, 64, 1)

In [25]:
model = Sequential()
# First Layer
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
BatchNormalization(axis=-1) # To normalize the activations
model.add(Activation('relu'))
# Second Layer
model.add(Conv2D(32, (3, 3)))
BatchNormalization(axis=-1)
model.add(Activation('relu'))
# Third Layer
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3, 3)))
BatchNormalization(axis=-1)
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
BatchNormalization(axis=-1)
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512))
BatchNormalization()
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(10))

model.add(Activation('softmax'))

In [26]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [30]:
train_data_shuffle = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, shear_range=0.4,height_shift_range=0.1, zoom_range=0.1)
test_data_shuffle = ImageDataGenerator()

In [31]:
train_generator = train_data_shuffle.flow(x_train_mnst_4d, y_train, batch_size=64)
test_generator = test_data_shuffle.flow(x_test_mnst_4d, y_test, batch_size=64)

In [32]:
# Let's see how it looks
x_test_mnst_4d.shape

(7000, 64, 64, 1)

In [33]:
model.fit_generator(train_generator, steps_per_epoch=28000//64, epochs=6,
                    validation_data=test_generator, validation_steps=7000//64)

Epoch 1/6
437/437 [==============================] - 413s - loss: 0.6617 - acc: 0.7933 - val_loss: 0.1440 - val_acc: 0.9586
Epoch 2/6
437/437 [==============================] - 395s - loss: 0.2611 - acc: 0.9256 - val_loss: 0.1198 - val_acc: 0.9674
Epoch 3/6
437/437 [==============================] - 391s - loss: 0.2031 - acc: 0.9426 - val_loss: 0.1142 - val_acc: 0.9704
Epoch 4/6
437/437 [==============================] - 403s - loss: 0.1798 - acc: 0.9476 - val_loss: 0.0984 - val_acc: 0.9748
Epoch 5/6
437/437 [==============================] - 398s - loss: 0.1553 - acc: 0.9549 - val_loss: 0.0940 - val_acc: 0.9749
Epoch 6/6
437/437 [==============================] - 420s - loss: 0.1431 - acc: 0.9588 - val_loss: 0.0879 - val_acc: 0.9808


In [37]:
score = model.evaluate(x_test_mnst_4d, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0817177257022
Test accuracy: 0.980142857075


In [34]:
kaggle_test_4d = kaggle_test.reshape(kaggle_test.shape[0], 64, 64, 1)
kaggle_test_4d = kaggle_test_4d.astype('float32')
kaggle_test_4d /= 255

In [35]:
pred = model.predict_classes(kaggle_test_4d)

500/500 [==============================] - 1s     
